In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
# The code was removed by Watson Studio for sharing.

,Negative_Review,Positive_Review,Reviewer_Score
0,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,2.9
1,No Negative,No real complaints the hotel was great great ...,7.5
2,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.1
3,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,3.8
4,You When I booked with your company on line y...,Amazing location and building Romantic setting,6.7


In [24]:
# We now convert the data into a spark dataframe to make use of the parellal processing benefits. We could also directly import the data as a Spark Datafram

reviews_spark = spark.createDataFrame(reviews_pd)
reviews_spark.show(5)

+--------------------+--------------------+--------------+
|     Negative_Review|     Positive_Review|Reviewer_Score|
+--------------------+--------------------+--------------+
| I am so angry th...| Only the park ou...|           2.9|
|         No Negative| No real complain...|           7.5|
| Rooms are nice b...| Location was goo...|           7.1|
| My room was dirt...| Great location i...|           3.8|
| You When I booke...| Amazing location...|           6.7|
| Backyard of the ...| Good restaurant ...|           6.7|
| Cleaner did not ...| The room is spac...|           4.6|
| Apart from the p...| Good location Se...|          10.0|
| Even though the ...|         No Positive|           6.5|
| The aircondition...| The room was big...|           7.9|
|  Nothing all great | Rooms were stunn...|          10.0|
| 6 30 AM started ...| Style location r...|           5.8|
| The floor in my ...| Comfy bed good l...|           4.6|
|         No Negative| This hotel is be...|           9.

In [25]:
# Before proceeding to feature transfromation, we first combine positive and negative review texts. Since our labels are based on both reviews. We can combine the words
# to build the model

from pyspark.sql.functions import concat, col, lit 

reviews_spark = reviews_spark.withColumn("Review",concat(col("Negative_Review"),lit(','),col("Positive_Review"))).drop("Negative_Review").drop("Positive_Review")
    
reviews_spark.show(5)

+--------------+--------------------+
|Reviewer_Score|              Review|
+--------------+--------------------+
|           2.9| I am so angry th...|
|           7.5|No Negative, No r...|
|           7.1| Rooms are nice b...|
|           3.8| My room was dirt...|
|           6.7| You When I booke...|
+--------------+--------------------+
only showing top 5 rows



In [26]:
# Our objective is to give words a positive or negative connation and in order to that we initially need to label our reviews. Instead of manually labelling each review,
# we shall use the given reviewer_score to give labels to our data.

# In order to give more clear labels, we will ignore rating between 3 to 6, and ratings less than 3 shall be given a negative label, while the ratings greater than 6
# shall be given a positive label

from pyspark.ml.feature import Bucketizer
splits = [0.0, 3, 7, 10]
bucketizer = Bucketizer(splits=splits, inputCol="Reviewer_Score", outputCol="Sentiment_signal")

# Transform original data into its bucket index.
bucketedData = bucketizer.transform(reviews_spark)

bucketedData = bucketedData.filter((bucketedData.Sentiment_signal != 1.0))
print("Bucketizer output with %d buckets" % (len(bucketizer.getSplits())-1))
bucketedData.show()

Bucketizer output with 3 buckets
+--------------+--------------------+----------------+
|Reviewer_Score|              Review|Sentiment_signal|
+--------------+--------------------+----------------+
|           2.9| I am so angry th...|             0.0|
|           7.5|No Negative, No r...|             2.0|
|           7.1| Rooms are nice b...|             2.0|
|          10.0| Apart from the p...|             2.0|
|           7.9| The aircondition...|             2.0|
|          10.0| Nothing all grea...|             2.0|
|           9.2|No Negative, This...|             2.0|
|           8.8| The staff in the...|             2.0|
|          10.0|No Negative, This...|             2.0|
|           7.5| We did not like ...|             2.0|
|           7.1|No Negative, Publ...|             2.0|
|           7.5| We had issues wi...|             2.0|
|           9.6| Transportation w...|             2.0|
|           9.6| Nothing , Lovely...|             2.0|
|           8.3| The bathroom in

In [28]:
# Now we go ahead and build our ML pipeline to transform our data into a set of vector that can be fed into an ML algorithm.

# This process will involve the following steps:

# First split sentences into tokens then remove stopwords being words that do not add lot of meaning to the sentence

# We then convert the toekns

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer, CountVectorizer, StopWordsRemover, StringIndexer

tokenizer = Tokenizer(inputCol="Review", outputCol="words")

remover = StopWordsRemover(inputCol="words", outputCol="words_clean")

cv = CountVectorizer(inputCol="words_clean", outputCol="features")

indexer = StringIndexer(inputCol = "Sentiment_signal", outputCol = "label")

pipeline = Pipeline(stages=[tokenizer, remover, cv, indexer])

pipelineFit = pipeline.fit(bucketedData)

transform_vector = pipelineFit.transform(bucketedData)

transform_vector.show(5)


+--------------+--------------------+----------------+--------------------+--------------------+--------------------+-----+
|Reviewer_Score|              Review|Sentiment_signal|               words|         words_clean|            features|label|
+--------------+--------------------+----------------+--------------------+--------------------+--------------------+-----+
|           2.9| I am so angry th...|             0.0|[, i, am, so, ang...|[, angry, made, p...|(78040,[0,1,4,5,1...|  1.0|
|           7.5|No Negative, No r...|             2.0|[no, negative,, n...|[negative,, real,...|(78040,[2,3,5,6,9...|  0.0|
|           7.1| Rooms are nice b...|             2.0|[, rooms, are, ni...|[, rooms, nice, e...|(78040,[0,2,3,4,5...|  0.0|
|          10.0| Apart from the p...|             2.0|[, apart, from, t...|[, apart, price, ...|(78040,[0,2,3,7,8...|  0.0|
|           7.9| The aircondition...|             2.0|[, the, aircondit...|[, aircondition, ...|(78040,[0,1,4,5,7...|  0.0|
+-------

In [30]:
from project_lib import Project
project = Project(sc,"b352b13d-9829-4fe1-8f30-961388e41c7e","p-422dcf71364664617425eea8fbac723fad534065")
project.save_data(file_name = "transform_vector.csv",data = transform_vector.toPandas().to_csv(index=False))
print("nope")

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 51724)
Traceback (most recent call last):
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/opt/ibm/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/opt/ibm/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/ibm/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updat

nope
